In [ ]:
import pandas as pd

# 1. Baca file CSV mentah
input_file = "item_tugas3.csv"
df = pd.read_csv(input_file, delimiter=";", skip_blank_lines=True)

# 2. Bersihkan nama kolom dari spasi
df.columns = [col.strip() for col in df.columns]

# 3. Hapus kolom kosong 
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.dropna(how='all', axis=1)

# 4. Fungsi bantu untuk membersihkan angka
def clean_numeric(val):
    if pd.isna(val):
        return 0
    val = str(val).replace("Rp", "").replace("Rp.", "")
    val = val.replace(".", "").replace(",", "").strip()
    return int(val) if val.isdigit() else val  # return value yg bkn numerik

def clean_float(val):
    if pd.isna(val):
        return 0.0
    val = str(val).replace("%", "").replace(",", ".").strip()
    try:
        return float(val)
    except:
        return val  # Return original value if conversion fails

# 5. Bersihkan kolom berdasarkan jenis data
for col in df.columns:
    # Periksa apakah kolom adalah kolom kategori
    if col.lower() == 'kategori':
        # Biarkan nilai kolom kategori apa adanya, hanya ganti NaN dengan string kosong
        df[col] = df[col].fillna('').astype(str)
    elif df[col].dtype == object:
        # Cek apakah kolom berisi data numerik dengan format tertentu
        if df[col].str.contains("Rp|%", na=False).any():
            if df[col].str.contains("%").any():
                df[col] = df[col].apply(clean_float)
            elif df[col].str.contains("Rp").any():
                df[col] = df[col].apply(clean_numeric)
        # Cek apakah kolom berisi angka dengan pemisah ribuan (.)
        elif df[col].str.contains(r"\d+\.\d+", na=False, regex=True).any():
            df[col] = df[col].apply(clean_numeric)
        else:
            # Jika kolom hanya berisi teks, biarkan tetap sebagai teks
            df[col] = df[col].astype(str)

# 6. Simpan ke file CSV baru
output_file = "cleaned_item.csv"
df.to_csv(output_file, index=False)

print(f"File hasil cleaning disimpan sebagai '{output_file}'")

File hasil cleaning disimpan sebagai 'cleaned_output.csv'
